In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from keras.initializers import glorot_uniform
from PIL import Image
import os



In [6]:
with tf.keras.utils.custom_object_scope({'GlorotUniform': glorot_uniform()}):
   model = keras.models.load_model('model_acc70.h5')

In [43]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 151, 85, 16)       448       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 76, 43, 16)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 25, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 13, 7, 32)         0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 4, 2, 64)          18496     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 2, 1, 64)          0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 128)              

In [33]:
img_path = os.path.abspath("./driving_dataset/34429.jpg")

In [34]:
display(Image(img_path))


TypeError: 'module' object is not callable

In [17]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array

In [20]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    
    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
        
         # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [25]:
img_size = (455, 256)


In [35]:
img_array = tf.keras.applications.resnet50.preprocess_input(get_img_array(img_path, size=img_size))


In [36]:
preds = model.predict(img_array)


In [38]:
decode_predictions = keras.applications.resnet50.decode_predictions


In [39]:
print("Predicted:", decode_predictions(preds, top=1)[0])


ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: (1, 10)

In [44]:
last_conv_layer_name= 'dense_35' 

In [45]:
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)


InvalidArgumentError: Invalid reduction dimension (2 for input with 2 dimension(s) [Op:Mean]